# Get Merge And View of MYSQL Database from ClickHouse Database
## Contents
1. [Introduction](#Introduction)
2. [Prerequisites](#Prerequisites)
3. [MySQL Database Operations](#MySQL-Database-Operations)
4. [ClickHouse Database Operations](#ClickHouse-Database-Operations)
5. [Comparison](#Comparison)

## Introduction
In this notebook, we will see how to get a View and Copy MySQL database from a ClickHouse database, create a MySQL database and a ClickHouse database, create a table in MySQL and two table in ClickHouse. 

Finally, we compare the data of the MySQL and ClickHouse view and copyied.

## Prerequisites
- ClickHouse Server
- MySQL Server
- MySQL Client
- ClickHouse Client

##### Note:
- follow the instructions in `README.md` to install the prerequisites.

### MySQL Database Operations

This part demonstrates operations related to MySQL databases, including creating a database, creating a table, and inserting data.

In [1]:
# Import necessary libraries
import mysql.connector
import time
from conf import *

#### Connect to MySQL Server
Connects to the MySQL server using the provided credentials.

In [2]:
# Connect to the MySQL server
mydb = mysql.connector.connect(
    host=MySQL_ip,
    user="root",
    password="1381",
    database=""
)

# Create a cursor object to execute SQL queries
cursor = mydb.cursor()

#### Create Database Function
Defines a function to create a new MySQL database or drop and recreate if it already exists.

In [3]:
# Function to create a database
def create_database():
    cursor.execute(f"DROP DATABASE IF EXISTS {DATABASE_}")
    print("Database dropped")
    mydb.commit()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DATABASE_}")
    print("Database created")
    mydb.commit()

#### Create Table Function
Defines a function to create a table within the specified database.

In [4]:
# Function to create a table
def create_table():
    cursor.execute(f"USE {DATABASE_}")
    cursor.execute(f"CREATE TABLE IF NOT EXISTS {TABLE_} (\
        `id` INT AUTO_INCREMENT PRIMARY KEY,\
        `name` VARCHAR(255),\
        `address` VARCHAR(255),\
        `email` VARCHAR(255),\
        `phone_number` VARCHAR(30),\
        `job` VARCHAR(255),\
        `company` VARCHAR(255),\
        `city` VARCHAR(255),\
        `country` VARCHAR(255),\
        `date` VARCHAR(255)\
    );")

#### Insert Data Real-time Function
Defines a function to insert data into the MySQL table in real-time.

In [5]:
# Function to insert data in real-time
def insert_data_real_time():
    with open('../mysql_data_directory/fake_data.csv', 'r') as file:
        next(file)  # Skip the first row

        for line in file:
            line = line.split(',')
            name = line[0]
            address = line[1]
            email = line[2]
            phone_number = line[3]
            job = line[4]
            company = line[5]
            city = line[6]
            country = line[7]
            date = line[8]
            sql = f"INSERT INTO {TABLE_} (name, address, email, phone_number, job, company, city, country, date) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
            val = (name, address, email, phone_number, job, company, city, country, date)
            cursor.execute(sql, val)
        mydb.commit()

#### Insert Data from CSV Function
Defines a function to insert data into the MySQL table from a CSV file.

In [6]:
# Function to insert data from CSV
def insert_data_csv():
    try:
        query = f"""
        LOAD DATA INFILE '/var/lib/mysql-files/fake_data.csv'
        INTO TABLE {TABLE_}
        FIELDS TERMINATED BY ','
        ENCLOSED BY '"'
        LINES TERMINATED BY '\n'
        IGNORE 1 LINES
        (name, address, email, phone_number, job, company, city, country, date);
        """
        cursor.execute(query)
        mydb.commit()
        print("Data inserted from CSV successfully.")
    except mysql.connector.Error as err:
        print(f"An error occurred: {err}")

#### Execute Database Operations
Calls the defined functions to create a database, create a table, and insert data into the table in real-time.

Ever time you run this cell, it will create a new database and table.

In [7]:
# Call the functions to create database and table
create_database()
create_table()


Database dropped
Database created


After creating the database and table, you can insert data into the table using the `Insert Data Real-time Function` or `Insert Data from CSV Function` functions.
Belwo is the code to insert data from CSV file containing 2 Million records.

#### Insert Data from CSV
You can run multiple times to insert more data.

In [8]:
for _ in range(5):
    insert_data_csv()


Data inserted from CSV successfully.
Data inserted from CSV successfully.
Data inserted from CSV successfully.
Data inserted from CSV successfully.
Data inserted from CSV successfully.


### ClickHouse Database Operations

This part demonstrates operations related to ClickHouse databases, including creating a database, creating tables, and copying data from MySQL to ClickHouse.

In [9]:
# Import necessary libraries
from clickhouse_driver import Client

#### Connect to ClickHouse Server
Connects to the ClickHouse server using the provided credentials.

In [10]:
# Connect to ClickHouse server
client = Client(
    host=ClickHouse_ip,
    port=9000,
)


#### Create Table Function
Defines a function to create a table within the specified database.

In [11]:
# Function to create ClickHouse database
def create_database_CH():
    query = f"DROP DATABASE IF EXISTS {DATABASE_};"
    client.execute(query)
    query = f"CREATE DATABASE IF NOT EXISTS {DATABASE_};"
    client.execute(query)
    query = f'USE {DATABASE_};'
    client.execute(query)


#### Copy Data from MySQL to ClickHouse Function
Defines a function to copy data from MySQL table to ClickHouse table.

In [12]:
# Function to copy table from MySQL to ClickHouse
def copy_table_from_mysql_to_clickhouse():
    # first we create the table in ClickHouse and get a View from the MySQL table
    # TABLE_ is the View name in ClickHouse
    query = f"CREATE TABLE IF NOT EXISTS {TABLE_} (\
        id UInt32,\
        name String,\
        address String,\
        email String,\
        phone_number String,\
        job String,\
        company String,\
        city String,\
        country String,\
        date String\
    ) ENGINE = MySQL('{MySQL_ip}:3306', '{DATABASE_}', '{TABLE_}', 'root', '1381');"
    client.execute(query)
    # NEWTABLE_ is the new table name in ClickHouse
    query = f"CREATE TABLE IF NOT EXISTS {DATABASE_}.{NEWTABLE_} (\
    id UInt32,\
    name String,\
    address String,\
    email String,\
    phone_number String,\
    job String,\
    company String,\
    city String,\
    country String,\
    date String\
    ) ENGINE = MergeTree() ORDER BY id;"
    client.execute(query)
    
    query = f"INSERT INTO {DATABASE_}.{NEWTABLE_} SELECT * FROM {DATABASE_}.{TABLE_};"
    client.execute(query)



#### Execute Database Operations
Calls the defined functions to create a database ClickHouse.

In [13]:
create_database_CH()

### Comparison
This part demonstrates how to compare the data of a MySQL and a ClickHouse view.

First we will create two Query, one for MySQL and one for ClickHouse 
##### Note: 
- You can change the query as per your requirement.

Then we will create a function to compare the data of the MySQL and ClickHouse view.

Finally, we will call the function to compare the data of the MySQL and ClickHouse view.


In [18]:
CH_query = f"SELECT count(DISTINCT name) FROM {DATABASE_}.{NEWTABLE_};"
CH_View_query = f"SELECT count(DISTINCT name) FROM {DATABASE_}.{TABLE_};"
MS_query = f"SELECT count(DISTINCT name) FROM {DATABASE_}.{TABLE_};"

##### Note:
- `MS_query` is the query to use for MYSQL.
- `CH_query` is the query to use for ClickHouse.
- `CH_View_query` is the query to use for ClickHouse View.

#### Run MySQL Query

In [22]:
start_time = time.time()
cursor.execute(MS_query)
cursor.fetchall()
end_time = time.time()
query_time_ms = (end_time - start_time) * 1000
print("Time taken for the query:", query_time_ms, "milliseconds")

Time taken for the query: 50929.23307418823 milliseconds


#### Run ClickHouse Query

In [20]:
start_time = time.time()
copy_table_from_mysql_to_clickhouse()
end_time = time.time()
query_time_ms = (end_time - start_time) * 1000
print( f"Time taken to copy table from MySQL to ClickHouse: {query_time_ms} ms")

start_time = time.time() * 1000  # Start time in milliseconds
result = client.execute(CH_query)
end_time = time.time() * 1000  # End time in milliseconds

execution_time_ms = end_time - start_time
print("Execution time:", execution_time_ms, "ms")

Time taken to copy table from MySQL to ClickHouse: 24543.262720108032 ms
Execution time: 1043.053955078125 ms


#### Run View ClickHouse Query

In [21]:
start_time = time.time() * 1000  # Start time in milliseconds
result = client.execute(CH_View_query)
end_time = time.time() * 1000  # End time in milliseconds

execution_time_ms = end_time - start_time
print("Execution time:", execution_time_ms, "ms")

Execution time: 4884.31591796875 ms
